In [1]:
import pandas as pd

raw_data = pd.read_csv("Chano's Cantina_search_results.csv")
data = raw_data.copy()
data.head()

,Unnamed: 0,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,0,6AJwsgXr7YwsqneGVAdgzw,las-catrinas-mexican-bar-and-eatery-astoria,Las Catrinas Mexican Bar & Eatery,https://s3-media3.fl.yelpcdn.com/bphoto/3uevye...,False,https://www.yelp.com/biz/las-catrinas-mexican-...,301,"[{'alias': 'mexican', 'title': 'Mexican'}, {'a...",4.0,"{'latitude': 40.7614214682633, 'longitude': -7...","['pickup', 'delivery']",$$,"{'address1': '32-02 Broadway', 'address2': '',...",1.917745e+10,(917) 745-0969,642.525771
1,1,jzVv_21473lAMYXIhVbuTA,de-mole-astoria-astoria,De Mole Astoria,https://s3-media1.fl.yelpcdn.com/bphoto/zZTTfd...,False,https://www.yelp.com/biz/de-mole-astoria-astor...,362,"[{'alias': 'mexican', 'title': 'Mexican'}]",4.0,"{'latitude': 40.7625999, 'longitude': -73.9129...","['pickup', 'delivery']",$$,"{'address1': '4220 30th Ave', 'address2': '', ...",1.718777e+10,(718) 777-1655,918.092772
2,2,2jjxdN9FXWVdx6DEX83krQ,hoja-santa-astoria,Hoja Santa,https://s3-media3.fl.yelpcdn.com/bphoto/xqo13E...,False,https://www.yelp.com/biz/hoja-santa-astoria?ad...,33,"[{'alias': 'newmexican', 'title': 'New Mexican...",4.5,"{'latitude': 40.7638197, 'longitude': -73.914523}",[],NaN,"{'address1': '40-17 30th Ave', 'address2': '',...",NaN,NaN,751.531469
3,3,AUyKmFjpaVLwc3awfUnqgQ,chela-and-garnacha-astoria,Chela & Garnacha,https://s3-media1.fl.yelpcdn.com/bphoto/ChVbA1...,False,https://www.yelp.com/biz/chela-and-garnacha-as...,364,"[{'alias': 'mexican', 'title': 'Mexican'}, {'a...",4.5,"{'latitude': 40.7557171543477, 'longitude': -7...","['pickup', 'delivery']",$$,"{'address1': '33-09 36th Ave', 'address2': '',...",1.917833e+10,(917) 832-6876,1318.326547
4,4,zNcDCPiUsE5aT6jyaCVtmA,frescos-cantina-long-island-city,Fresco's Cantina,https://s3-media2.fl.yelpcdn.com/bphoto/3IdlTT...,False,https://www.yelp.com/biz/frescos-cantina-long-...,209,"[{'alias': 'mexican', 'title': 'Mexican'}, {'a...",4.5,"{'latitude': 40.76837, 'longitude': -73.93303}","['restaurant_reservation', 'pickup', 'delivery']",$$,"{'address1': '12-14 31st Ave', 'address2': Non...",1.347809e+10,(347) 808-7338,927.218713


In [3]:
cols = ['name', 'url', 'review_count', 'categories', 'rating', 'price']
df = data[cols]

df.head()

,name,url,review_count,categories,rating,price
0,Las Catrinas Mexican Bar & Eatery,https://www.yelp.com/biz/las-catrinas-mexican-...,301,"[{'alias': 'mexican', 'title': 'Mexican'}, {'a...",4.0,$$
1,De Mole Astoria,https://www.yelp.com/biz/de-mole-astoria-astor...,362,"[{'alias': 'mexican', 'title': 'Mexican'}]",4.0,$$
2,Hoja Santa,https://www.yelp.com/biz/hoja-santa-astoria?ad...,33,"[{'alias': 'newmexican', 'title': 'New Mexican...",4.5,NaN
3,Chela & Garnacha,https://www.yelp.com/biz/chela-and-garnacha-as...,364,"[{'alias': 'mexican', 'title': 'Mexican'}, {'a...",4.5,$$
4,Fresco's Cantina,https://www.yelp.com/biz/frescos-cantina-long-...,209,"[{'alias': 'mexican', 'title': 'Mexican'}, {'a...",4.5,$$


In [26]:
import ast
tags=[]

for cat in df['categories']:
    cat = ast.literal_eval(cat)
    for tag in cat:
        tags.append(tag['title'])
        
tags = list(set(tags))
for tag in tags:
    df[tag]=0
    
df.head()

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


,name,url,review_count,categories,rating,price,Cocktail Bars,Tacos,Breakfast & Brunch,Salad,...,Tapas/Small Plates,Pizza,Wine Bars,Bars,Tapas Bars,Southern,Lounges,Mexican,Empanadas,Food Stands
0,Las Catrinas Mexican Bar & Eatery,https://www.yelp.com/biz/las-catrinas-mexican-...,301,"[{'alias': 'mexican', 'title': 'Mexican'}, {'a...",4.0,$$,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,De Mole Astoria,https://www.yelp.com/biz/de-mole-astoria-astor...,362,"[{'alias': 'mexican', 'title': 'Mexican'}]",4.0,$$,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Hoja Santa,https://www.yelp.com/biz/hoja-santa-astoria?ad...,33,"[{'alias': 'newmexican', 'title': 'New Mexican...",4.5,NaN,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Chela & Garnacha,https://www.yelp.com/biz/chela-and-garnacha-as...,364,"[{'alias': 'mexican', 'title': 'Mexican'}, {'a...",4.5,$$,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Fresco's Cantina,https://www.yelp.com/biz/frescos-cantina-long-...,209,"[{'alias': 'mexican', 'title': 'Mexican'}, {'a...",4.5,$$,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
for ix in df.index:
    cat = ast.literal_eval(df['categories'][ix])
    for tag in cat:
        col = tag['title']
        df[col][ix] = 1

df.head()

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
